In [1]:
#imports
import pandas as pd
import numpy as np
import os
import time
import matplotlib.pyplot as plt
import random

In [2]:
path_ground_truth = r"..\groundTruth"
path_myo_data = r"..\MyoData"

userGT_list = os.listdir(path_ground_truth)
userMyo_list = os.listdir(path_myo_data)
#By the assignment details will be using EMG data not IMU 


In [3]:
#Use for later iteration through the user list, will test on first user, user09
userGT_list = [path_ground_truth + "\\" + i for i in userGT_list]
userMyo_list = [path_myo_data + "\\" + i for i in userMyo_list]


In [4]:
myo_fork_user = [userMyo_list[0] + "\\fork\\" + i for i in os.listdir(userMyo_list[0] + "\\fork")] 
myo_spoon_user = [userMyo_list[0] + "\\spoon\\" + i for i in os.listdir(userMyo_list[0] + "\\spoon")] 
femg = pd.read_csv(myo_fork_user[0],
                           names = ["UNIX_TIME","EMG1","EMG2",
                                    "EMG3","EMG4","EMG5",
                                    "EMG6","EMG7","EMG8"])

semg = pd.read_csv(myo_spoon_user[0],
                  names = ["UNIX_TIME","EMG1","EMG2",
                            "EMG3","EMG4","EMG5",
                            "EMG6","EMG7","EMG8"])

ground_fork_user = [userGT_list[0] + "\\fork\\" + i for i in os.listdir(userGT_list[0] + "\\fork")] 
ground_spoon_user = [userGT_list[0] + "\\spoon\\" + i for i in os.listdir(userGT_list[0] + "\\spoon")]

forkgt = pd.read_csv(ground_fork_user[0], names = ["Start", "End","Ignore"])
spoongt = pd.read_csv(ground_spoon_user[0], names = ["Start", "End","Ignore"])

In [5]:
spoongt = spoongt[["Start","End"]]
forkgt = forkgt[["Start","End"]]

In [6]:
forkgt.head()

,Start,End
0,234,280
1,781,860
2,1220,1258
3,1570,1607
4,2096,2126


In [7]:
#convert IMU and EMG to seconds to get an idea of sample rate
average = 0
count = 0
for i in range(1000):
    diff = int(femg[i+1:i+2]["UNIX_TIME"])/1000 - int(femg[i:i+1]["UNIX_TIME"])/1000
    average = average + diff
    count = count + 1
average = average/count
print("EMG sample rate: " + str(average) + " " + str(count))
#average shows that sample rate is about 100hz for emg and imu is 50hz

EMG sample rate: 0.009756999969482422 1000


In [8]:
#Normalize the data between ground truth framerate (30f/sec) and emg data (100hzs = 100/s)

#fork ground truth for eating and non-eating normalized to 100hz
fgt_eating = np.round_((forkgt.values * 100 / 30))

fgt_noneating = []
last_stop = 0
for i in fgt_eating:
    if last_stop == 0:
        fgt_noneating.append([0,i[0]]) 
    else:
        fgt_noneating.append([last_stop,i[0]])
    last_stop = i[1]

#Spoon ground truth for eating and non-eating, normalized to 100hz
sgt_eating = np.round_((spoongt.values * 100 / 30))

sgt_noneating = []
last_stop = 0
for i in sgt_eating:
    if last_stop == 0:
        sgt_noneating.append([0,i[0]]) 
    else:
        sgt_noneating.append([last_stop,i[0]])
    last_stop = i[1]

In [9]:
#Now with normalized data and the idea that eating and sensor data cn be assumed to start at 0 for ground truth data
#We can splice out eat from non-eating as the ground truth data has essentially been turned into ranges for EMG data

#Fork non-eating data frame creation
femg_noneating = pd.DataFrame()
for i in range(len(fgt_noneating)):
    temp = femg[int(fgt_noneating[i][0]):int(fgt_noneating[i][1])] 
    femg_noneating = pd.concat([femg_noneating,temp])

#Fork eating data frame creation 
femg_eating = pd.DataFrame()
for i in range(len(fgt_eating)):
    temp = femg[int(fgt_eating[i][0]):int(fgt_eating[i][1])] 
    femg_eating = pd.concat([femg_eating,temp])

    
#Spoon non-eating data frame creation 
semg_noneating = pd.DataFrame()
for i in range(len(sgt_noneating)):
    temp = semg[int(sgt_noneating[i][0]):int(sgt_noneating[i][1])] 
    semg_noneating = pd.concat([semg_noneating,temp])

#Spoon eating data frame creation     
semg_eating = pd.DataFrame()
for i in range(len(sgt_eating)):
    temp = semg[int(sgt_eating[i][0]):int(sgt_eating[i][1])] 
    semg_eating = pd.concat([semg_eating,temp])

In [19]:
femg # data frame for all fork emg data

,UNIX_TIME,EMG1,EMG2,EMG3,EMG4,EMG5,EMG6,EMG7,EMG8
0,1503510449719,-1,-2,-1,-3,-2,-1,2,0
1,1503510449726,0,0,0,-2,0,-1,0,0
2,1503510449741,-3,0,0,-1,-2,-1,-1,-1
3,1503510449749,0,1,-1,-1,-3,-2,-1,0
4,1503510449756,0,-3,-3,0,0,0,-4,-2
5,1503510449772,1,-1,0,0,3,0,1,-1
6,1503510449781,1,0,-1,-1,-1,-4,-6,-1
7,1503510449786,-1,-3,-1,-1,-1,-3,-2,0
8,1503510449803,-1,0,0,-1,-1,2,0,1
9,1503510449809,0,-1,-1,-2,-1,5,-1,0


In [10]:
 #data frame for all fork data that is classified as eating
femg_eating['eating'] = [True for i in range(len(femg_eating))] #Add in the classification variable of 'eating' set to true
femg_noneating['eating'] = [False for i in range(len(femg_noneating))]

semg_eating['eating'] = [True for i in range(len(semg_eating))] #Add in the classification variable of 'eating' set to true
semg_noneating['eating'] = [False for i in range(len(semg_noneating))]

In [24]:
femg_eating

,UNIX_TIME,EMG1,EMG2,EMG3,EMG4,EMG5,EMG6,EMG7,EMG8,eating
780,1503510457338,5,38,12,-2,0,0,4,10,True
781,1503510457352,-2,-5,-9,-47,-8,-4,-9,15,True
782,1503510457361,-3,-15,-7,6,2,-2,-5,-18,True
783,1503510457368,9,-6,-2,13,2,5,-1,-7,True
784,1503510457383,-4,2,0,-11,-5,-2,-1,7,True
785,1503510457389,-16,-4,-8,-3,0,-1,1,13,True
786,1503510457399,-10,13,3,-1,-1,-22,-4,-7,True
787,1503510457412,-7,-18,-2,10,-1,-8,5,0,True
788,1503510457420,-19,0,11,2,0,2,-5,12,True
789,1503510457428,5,5,7,7,3,3,8,21,True


In [19]:
#Next step, equalizing the number of eating and non-eating samples, 
#will randomly sample out noneating samples for as many eating samples
femg_noneating_sub = femg_noneating.sample(len(femg_eating),replace=False) #Fork non-eating subset
semg_noneating_sub = semg_noneating.sample(len(semg_eating),replace=False) #Spoon non-eating subset

In [20]:
femg_noneating.describe() # data frame for all fork non-eating

,UNIX_TIME,EMG1,EMG2,EMG3,EMG4,EMG5,EMG6,EMG7,EMG8
count,2.301500e+04,23015.000000,23015.000000,23015.000000,23015.000000,23015.000000,23015.000000,23015.000000,23015.000000
mean,1.503511e+12,-0.687465,-0.799131,-0.833804,-0.763676,-0.777145,-0.758549,-0.767586,-0.799783
std,7.126162e+04,12.696176,8.863228,3.563666,3.077443,2.984112,2.703500,3.068678,7.071865
min,1.503510e+12,-94.000000,-77.000000,-28.000000,-74.000000,-98.000000,-82.000000,-28.000000,-86.000000
25%,1.503511e+12,-7.000000,-5.000000,-3.000000,-2.000000,-2.000000,-2.000000,-2.000000,-3.000000
50%,1.503511e+12,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
75%,1.503511e+12,6.000000,3.000000,1.000000,0.000000,0.000000,0.000000,1.000000,2.000000
max,1.503511e+12,88.000000,82.000000,54.000000,91.000000,127.000000,92.000000,31.000000,70.000000


In [21]:
femg_noneating_sub.describe()

,UNIX_TIME,EMG1,EMG2,EMG3,EMG4,EMG5,EMG6,EMG7,EMG8
count,2.308000e+03,2308.000000,2308.000000,2308.000000,2308.000000,2308.000000,2308.000000,2308.000000,2308.000000
mean,1.503511e+12,-0.484835,-1.081023,-1.001733,-0.794627,-0.856153,-0.847487,-0.827990,-0.685009
std,7.128842e+04,12.924851,8.576382,3.459468,3.293125,3.194208,2.928147,3.129534,6.885842
min,1.503510e+12,-84.000000,-42.000000,-22.000000,-42.000000,-85.000000,-82.000000,-25.000000,-53.000000
25%,1.503511e+12,-7.000000,-6.000000,-3.000000,-2.000000,-2.000000,-2.000000,-2.000000,-3.000000
50%,1.503511e+12,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
75%,1.503511e+12,6.000000,3.000000,1.000000,0.000000,0.000000,0.000000,1.000000,2.000000
max,1.503511e+12,81.000000,60.000000,17.000000,91.000000,59.000000,25.000000,13.000000,48.000000


In [12]:
femg_eating.describe()

,UNIX_TIME,EMG1,EMG2,EMG3,EMG4,EMG5,EMG6,EMG7,EMG8
count,2.308000e+03,2308.000000,2308.000000,2308.000000,2308.000000,2308.000000,2308.000000,2308.000000,2308.000000
mean,1.503511e+12,-0.816724,-0.837522,-0.844887,-0.778596,-0.792894,-0.743934,-0.764298,-0.813692
std,7.250578e+04,13.183271,9.126621,3.742148,3.459084,1.717475,2.392524,3.567771,8.462772
min,1.503510e+12,-106.000000,-53.000000,-17.000000,-47.000000,-20.000000,-27.000000,-29.000000,-53.000000
25%,1.503511e+12,-8.000000,-6.000000,-3.000000,-2.000000,-2.000000,-2.000000,-2.000000,-4.000000
50%,1.503511e+12,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
75%,1.503511e+12,6.000000,3.000000,1.000000,0.000000,0.000000,0.000000,1.000000,2.000000
max,1.503511e+12,63.000000,68.000000,38.000000,47.000000,18.000000,27.000000,24.000000,92.000000


In [51]:
semg # data frame for all spoon emg data 34576 rows

,UNIX_TIME,EMG1,EMG2,EMG3,EMG4,EMG5,EMG6,EMG7,EMG8
0,1503510957307,-1,-1,0,-3,0,-2,-4,-4
1,1503510957314,-1,-6,-23,-10,-1,-1,-2,-4
2,1503510957322,1,0,1,1,0,-1,-2,2
3,1503510957336,-2,3,-3,0,-1,-1,1,-2
4,1503510957348,-2,-1,-4,-2,1,1,2,1
...,...,...,...,...,...,...,...,...,...
34571,1503511293211,-4,2,-2,-2,0,-1,-2,0
34572,1503511293216,0,0,0,-2,-3,1,-2,-1
34573,1503511293231,-1,-3,-4,-1,-1,0,2,3
34574,1503511293239,-6,-2,2,-1,0,0,1,1


In [52]:
semg_noneating #data frame for all spoon eating data, spliced out 31251 rows

,UNIX_TIME,EMG1,EMG2,EMG3,EMG4,EMG5,EMG6,EMG7,EMG8
0,1503510957307,-1,-1,0,-3,0,-2,-4,-4
1,1503510957314,-1,-6,-23,-10,-1,-1,-2,-4
2,1503510957322,1,0,1,1,0,-1,-2,2
3,1503510957336,-2,3,-3,0,-1,-1,1,-2
4,1503510957348,-2,-1,-4,-2,1,1,2,1
...,...,...,...,...,...,...,...,...,...
34571,1503511293211,-4,2,-2,-2,0,-1,-2,0
34572,1503511293216,0,0,0,-2,-3,1,-2,-1
34573,1503511293231,-1,-3,-4,-1,-1,0,2,3
34574,1503511293239,-6,-2,2,-1,0,0,1,1


In [53]:
semg_eating #data frame for all spoon eating data spliced out to 3325 rows

,UNIX_TIME,EMG1,EMG2,EMG3,EMG4,EMG5,EMG6,EMG7,EMG8
780,1503510964844,-4,0,0,9,0,-1,-2,1
781,1503510964859,14,2,-3,-2,1,1,-2,-1
782,1503510964867,7,-3,-5,-4,-2,-3,-1,1
783,1503510964874,-3,-5,3,-1,-2,0,-2,-3
784,1503510964889,-3,-6,-7,-3,-1,-1,-2,-1
...,...,...,...,...,...,...,...,...,...
33718,1503511284938,-10,0,2,-3,-2,-4,-2,-6
33719,1503511284952,-1,1,-2,-2,0,-2,-1,0
33720,1503511284958,3,5,1,1,-1,0,1,1
33721,1503511284967,10,-9,-3,-3,-1,-2,-1,-3


In [ ]:
femg_noneating_sub.describe()


In [66]:
femg_eating



,UNIX_TIME,EMG1,EMG2,EMG3,EMG4,EMG5,EMG6,EMG7,EMG8
6772,1503511023227,-28,-3,-1,-2,-1,1,0,-3
8956,1503511044423,5,30,12,0,1,0,5,18
4109,1503510997193,8,3,-1,0,-1,-3,1,2
18731,1503511139323,-3,4,-1,-2,0,-1,-1,-3
3874,1503510994992,4,0,0,-1,-1,0,0,-4
...,...,...,...,...,...,...,...,...,...
29885,1503511247738,15,4,0,1,0,9,4,-6
1202,1503510968968,13,-3,-11,-2,-2,-5,-5,-13
3416,1503510990493,14,-7,-2,0,-1,-1,1,0
15292,1503511106009,2,2,-3,-6,-2,1,-2,0
